### Import Libraries

In [41]:
import os, json, textwrap, re, time
import pandas as pd
import tqdm

import requests 

### Constants

In [9]:
API_KEY  = os.getenv("OPENAI_API_KEY", "cse476")
API_BASE = os.getenv("API_BASE", "http://10.4.58.53:41701/v1")  
MODEL    = os.getenv("MODEL_NAME", "bens_model") 

In [74]:
def call_model(prompt: str,
               system: str = "You are a helpful assistant. Reply with only the final answer—no explanation.",
               model: str = MODEL,
               temperature: float = 0.0,
               timeout: int = 60) -> dict:
    """
    Calls an OpenAI-style /v1/chat/completions endpoint and returns:
    { 'ok': bool, 'text': str or None, 'raw': dict or None, 'status': int, 'error': str or None, 'headers': dict }
    """
    url = f"{API_BASE}/chat/completions"
    headers = {
        "Authorization": f"Bearer {API_KEY}",
        "Content-Type":  "application/json",
    }
    payload = {
        "model": model,
        "messages": [
            {"role": "system", "content": system},
            {"role": "user",   "content": prompt}
        ],
        "temperature": temperature,
        "max_tokens": 1024,
    }

    try:
        response = requests.post(url, headers=headers, json=payload, timeout=timeout)
        status = response.status_code
        hdrs   = dict(response.headers)
        if status == 200:
            data = response.json()
            text = data.get("choices", [{}])[0].get("message", {}).get("content", "")
            return {"ok": True, "text": text, "raw": data, "status": status, "error": None, "headers": hdrs}
        else:
            # try best-effort to surface error text
            err_text = None
            try:
                err_text = response.json()
            except Exception:
                err_text = response.text
            return {"ok": False, "text": None, "raw": None, "status": status, "error": str(err_text), "headers": hdrs}
    except requests.RequestException as e:
        return {"ok": False, "text": None, "raw": None, "status": -1, "error": str(e), "headers": {}}


In [4]:
PATH = '/Users/porter/Programming/final_project_tutorial_and_dev_data/cse476_final_project_dev_data.json'

with open(PATH, 'r') as file:
    dataset= pd.DataFrame(json.load(file))

In [13]:
dataset['domain'].value_counts()

domain
common_sense         400
math                 300
coding               100
future_prediction    100
planning             100
Name: count, dtype: int64

In [39]:
math_dataset=dataset.loc[dataset['domain']=='math'].reset_index()
math_dataset.head()

,index,input,output,domain
0,0,Let $ABCD$ be a convex quadrilateral with $AB ...,112,math
1,1,A tennis player computes her win ratio by divi...,164,math
2,2,What is the product of the real roots of the e...,20,math
3,3,"In $\triangle ABC$ , $AB= 425$ , $BC=450$ , an...",306,math
4,4,How many even integers between 4000 and 7000 h...,728,math


In [ ]:
model_output=[]
SYSTEM='''
You are Qwen, created by Alibaba Cloud. You are a helpful assistant.
Explore multiple angles and approaches. Continuously adjust your reasoning based on intermediate results and reflections, adapting your strategy as you progress. Regularly evaluate progress. Be critical and honest about your own reasoning process but do not tell me about it.

Reply with only the final answer NO other explanation or any other text is needed. 

Q: Solve for the smallest integer n such that 3n + 5 > 26.

Answer: 8

Q: In a race, you pass the person in second place. What position are you now in?

Answer: second 

'''
for question in tqdm.tqdm(math_dataset['input']):
    response=call_model(prompt=f'Q: {question}',system=SYSTEM)
    answer=re.search(r'[-+]?\d+(\.\d+)?', response['text'])
    
    if not answer:
        model_output.append(None)
        continue

    model_output.append(answer.group())

math_dataset['response']=model_output

100%|██████████| 300/300 [01:22<00:00,  3.62it/s]


In [47]:
math_dataset.to_csv('dataset/base_math_result.csv')

In [51]:
print(f'Correct: {len(math_dataset.loc[math_dataset['output']==math_dataset['response']])}')
print(f'Incorrect: {len(math_dataset.loc[math_dataset['output']!=math_dataset['response']])}')

Correct: 21
Incorrect: 279


In [62]:
math_dataset.loc[math_dataset['response'].isna()]

,index,input,output,domain,response
204,604,The results of a cross-country team's training...,\text{Evelyn},math,None
297,697,A figure skater is facing north when she begin...,\text{east},math,None


In [ ]:
math_dataset['input'].iloc[297]

'A figure skater is facing north when she begins to spin to her right. She spins 2250 degrees. Which direction (north, south, east or west) is she facing when she finishes her spin?'

In [89]:
PROMPT='Let $ABCD$ be a convex quadrilateral with $AB = CD = 10$ , $BC = 14$ , and $AD = 2\\sqrt{65}$ . Assume that the diagonals of $ABCD$ intersect at point $P$ , and that the sum of the areas of triangles $APB$ and $CPD$ equals the sum of the areas of triangles $BPC$ and $APD$ . Find the area of quadrilateral $ABCD$ .'

SYSTEM='''
You are Qwen, created by Alibaba Cloud. You are a helpful assistant.

Begin by enclosing all thoughts within <thinking> tags, exploring multiple angles and approaches. Break down the solution into clear steps within <step> tags. Start with a 10 step budget, requesting more for complex problems if needed. Use <count> tags after each step to show the remaining budget. Stop when reaching 0. Continuously adjust your reasoning based on intermediate results and reflections, adapting your strategy as you progress. Regularly evaluate progress using <reflection> tags. Be critical and honest about your reasoning process. Assign a quality score between 0.0 and 1.0 using <reward> tags after each reflection. Use this to guide your approach: 0.8+: Continue current approach 0.5-0.7: Consider minor adjustments Below 0.5: Seriously consider backtracking and trying a different approach If unsure or if reward score is low, backtrack and try a different approach, explaining your decision within <thinking> tags. For mathematical problems, show all work explicitly using LaTeX for formal notation and provide detailed proofs. Explore multiple solutions individually if possible, comparing approaches in reflections. Use thoughts as a scratchpad, writing out all calculations and reasoning explicitly. Synthesize the final answer within <answer> tags, providing a clear, concise summary. Conclude with a final reflection on the overall solution, discussing effectiveness, challenges, and solutions. Assign a final reward score.
'''
test=call_model(prompt=PROMPT,system=SYSTEM)
print(test['text'])

<thinking>
Let’s analyze the problem step by step.

We are given a convex quadrilateral $ABCD$ with the following side lengths:
- $AB = 10$
- $CD = 10$
- $BC = 14$
- $AD = 2\sqrt{65}$

The diagonals $AC$ and $BD$ intersect at point $P$, and it is given that:
$$
\text{Area}(\triangle APB) + \text{Area}(\triangle CPD) = \text{Area}(\triangle BPC) + \text{Area}(\triangle APD)
$$

This implies that the diagonals divide the quadrilateral into four triangles of equal total area, i.e., the sum of the areas of the two opposite triangles is equal.

Let’s denote the area of quadrilateral $ABCD$ as $K$. Then, since the diagonals divide the quadrilateral into four triangles, and the sum of the areas of the two opposite triangles is equal, we can say:
$$
\text{Area}(\triangle APB) + \text{Area}(\triangle CPD) = \text{Area}(\triangle BPC) + \text{Area}(\triangle APD) = \frac{K}{2}
$$

So, the total area of the quadrilateral is:
$$
K = 2 \times \left(\text{Area}(\triangle APB) + \text{Area}(\triangle

In [88]:
test

{'ok': True,
 'text': 'We are given a convex quadrilateral $ABCD$ with the following side lengths:\n\n- $AB = 10$\n- $CD = 10$\n- $BC = 14$\n- $AD = 2\\sqrt{65}$\n\nAlso, the diagonals $AC$ and $BD$ intersect at point $P$, and it is given that:\n\n$$\n\\text{Area of } \\triangle APB + \\text{Area of } \\triangle CPD = \\text{Area of } \\triangle BPC + \\text{Area of } \\triangle APD\n$$\n\nThis condition implies that the **diagonals divide the quadrilateral into four triangles of equal total area**, and the **sum of the areas of opposite triangles is equal**. This is a **key geometric property** that tells us that the diagonals **bisect each other in area**, i.e., the diagonals **divide the quadrilateral into four triangles of equal area**.\n\nThis is a **special property** of **parallelograms**, but it is **not necessarily a parallelogram**. However, this condition is **sufficient to conclude that the area of the quadrilateral is twice the area of any of the four triangles**.\n\nLet’s

In [86]:
test

{'ok': True,
 'text': 'We are given a convex quadrilateral $ABCD$ with the following side lengths:\n\n- $AB = 10$\n- $CD = 10$\n- $BC = 14$\n- $AD = 2\\sqrt{65}$\n\nLet the diagonals $AC$ and $BD$ intersect at point $P$. It is also given that:\n\n$$\n\\text{Area}(\\triangle APB) + \\text{Area}(\\triangle CPD) = \\text{Area}(\\triangle BPC) + \\text{Area}(\\triangle APD)\n$$\n\nWe are to find the **area of quadrilateral $ABCD$**.\n\n---\n\n### Step 1: Use the given area condition\n\nLet’s denote the areas of the four triangles formed by the diagonals as follows:\n\n- $\\text{Area}(\\triangle APB) = A_1$\n- $\\text{Area}(\\triangle BPC) = A_2$\n- $\\text{Area}(\\triangle CPD) = A_3$\n- $\\text{Area}(\\triangle APD) = A_4$\n\nFrom the problem, we are given:\n\n$$\nA_1 + A_3 = A_2 + A_4\n$$\n\nThis implies:\n\n$$\nA_1 + A_3 = A_2 + A_4 \\Rightarrow A_1 + A_3 = A_2 + A_4\n$$\n\nNow, the total area of quadrilateral $ABCD$ is:\n\n$$\n\\text{Area}(ABCD) = A_1 + A_2 + A_3 + A_4\n$$\n\nFrom the 